# Градиентный бустинг

Бустинг это метод построения компизиции алгоритмов, в котором базовые алгоритмы строятся последовательно один за другим, причем каждый следующий алгоритм строится таким образом, чтобы уменьшить ошибку предыдущего.

Положим, что алгоритм это линейная комбинация некоторых базовых алгоритмов:
    $$a_N(x) = \sum_{n=1}^N b_n(x)$$

Пусть задана некоторая функция потетерь, которую мы оптимизируем
$$\sum_{i=1}^l L(\hat y_i, y_i) \rightarrow min$$ 


Зададимся вопросом: а что если мы хотим добавить ещё один алгоритм в эту композицию, но не просто добавить, а как можно оптимальнее с точки зрения исходной оптимизационной задачи. То есть уже есть какой-то алгоритм $a_N(x)$ и мы хотим прибавить к нему базовый алгоритм $b_{N+1}(x)$:

$$\sum_{i=1}^l L(a_{N}(x_i) + b_{N+1}(x_i), y_i) \to \min_{b_{N+1}}$$

Сначала имеет смысл решить более простую задачу: определить, какие значения $r_1 ,r_2 ..., r_l$ должен принимать алгоритм $b_N(x_i) = r_i$ на объектах обучающей выборки, чтобы ошибка на обучающей выборке была минимальной:

$$F(r) = \sum_{i=1}^l L(a_{N}(x_i) + r_i, y_i) \to \min_{r},$$

где $r = (r_1, r_2, \dots, r_l)$ - вектор сдвигов.

Поскольку направление наискорейшего убывания функции задается направлением антиградиента, его можно принять в качестве вектора $r$:
$$r = -\nabla F \\$$
$$r_i = \frac{\partial{L}(a_N(x_i), y_i))}{\partial{a_N(x_i)}}, \ \ \ i = \overline{1,l}$$

Компоненты вектора $r$, фактически, являются теми значениями, которые на объектах обучающей выборки должен принимать новый алгоритм $b_{N+1}(x)$, чтобы минимизировать ошибку строящейся композиции. 
Обучение $b_{N+1}(x)$, таким образом, представляет собой *задачу обучения на размеченных данных*, в которой ${(x_i , r_i )}_{i=1}^l$ — обучающая выборка, и используется, например, квадратичная функция ошибки:
$$b_{N+1}(x) = arg \min_{b}\sum_{i=1}^l(b(x_i) - r_i)^2$$

Таким образом, можно подобрать неплохое улучшение текущего алгоритма $a_N(x)$, а потом ещё раз и ещё, в итоге получив комбинацию алгоритмов, которая будет минимизировать исходный функционал.

# Бустинг над решающими деревьями

Наиболее популярное семейство алгоритмов для бустинга это деревья. Рассмотрим популярные библиотеки

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score, train_test_split

from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier

import warnings
warnings.filterwarnings('ignore')

In [2]:
data = pd.read_csv('HR.csv')

In [3]:
data.head()

,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years
0,0.53,2,157,3,0,1,0
1,0.86,5,262,6,0,0,0
2,0.88,7,272,4,0,1,0
3,0.87,5,223,5,0,1,0
4,0.52,2,159,3,0,1,0


In [4]:
X, y = data.drop('left', axis=1).values, data['left'].values

Качество классификации решающим деревом с настройками по-умолчанию:

In [5]:
%%time
print("XGBClassifier: {:.4f}".format(cross_val_score(XGBClassifier(), X, y).mean()))

XGBClassifier: 0.7791
CPU times: user 1.05 s, sys: 4.04 ms, total: 1.06 s
Wall time: 1.06 s


In [6]:
%%time
print("CatBoostClassifier: {:.4f}".format(cross_val_score(CatBoostClassifier(verbose=False), X, y).mean()))

CatBoostClassifier: 0.7776
CPU times: user 1min 45s, sys: 52.7 s, total: 2min 38s
Wall time: 50.4 s


In [7]:
%%time
print("LGBMClassifier: {:.4f}".format(cross_val_score(LGBMClassifier(), X, y).mean()))

LGBMClassifier: 0.7790
CPU times: user 562 ms, sys: 24.8 ms, total: 587 ms
Wall time: 586 ms


## Опциональное задание
Поиграйтесь с основными параметрами алгоритмов, чтобы максимизировать качество